In [1]:
from csense.benchmark import cqa
from PyDictionary import PyDictionary
import spacy
import nltk

dictionary=PyDictionary()
# import en_core_web_sm

# nlp = en_core_web_sm.load()
# nlp = spacy.load("/Users/anushkabaoni/Downloads/Cornell Courses-Spring2020/Advanced AI/CommonsenseQADataset/venv/lib/python3.7/site-packages/en_core_web_sm")
# nlp = spacy.load("../CommonsenseQADataset/venv/lib/python3.7/site-packages/en_core_web_sm")
nlp = spacy.load("en_core_web_sm")

In [2]:
dataset = cqa.getDataSet()

In [3]:
what_questions = []
where_questions = []
why_questions = []
who_questions = []
which_questions = []
when_questions = []
how_questions = []

rand = []

for question in dataset:
    assigned = False
    text = set(nltk.word_tokenize(question['question']['stem'].lower()))
    if("what" in text and "where" not in text and "why" not in text and "who" not in text and "when" not in text and "how" not in text and "which" not in text):
        what_questions.append(question)
        continue
    if("what" not in text and "where" in text and "why" not in text and "who" not in text and "when" not in text and "how" not in text and "which" not in text):
        where_questions.append(question)
        continue
    if("what" not in text and "where" not in text and "why" in text and "who" not in text and "when" not in text and "how" not in text and "which" not in text):
        why_questions.append(question)
        continue
    if("what" not in text and "where" not in text and "why" not in text and "who" in text and "when" not in text and "how" not in text and "which" not in text):
        who_questions.append(question)
        continue
    if("what" not in text and "where" not in text and "why" not in text and "who" not in text and "when" in text and "how" not in text and "which" not in text):
        when_questions.append(question)
        continue
    if("what" not in text and "where" not in text and "why" not in text and "who" not in text and "when" not in text and "how" in text and "which" not in text):
        how_questions.append(question)
        continue     
    if("what" not in text and "where" not in text and "why" not in text and "who" not in text and "when" not in text and "how" not in text and "which" in text):
        which_questions.append(question)
        continue        
    # whether, whose, whom, how
    
    doc = nlp(question['question']['stem'].lower())
    for token in doc:
        if(token.dep_ == 'ROOT'):
            childern = [str(child) for child in token.children]         
            if("what" in childern):
                what_questions.append(question)
                assigned=True
                break
            elif("where" in childern):
                where_questions.append(question)
                assigned=True
                break
            elif("why" in childern):
                why_questions.append(question)
                assigned=True
                break
            elif("who" in childern):
                who_questions.append(question)
                assigned=True
                break
            elif("how" in childern):
                how_questions.append(question)
                assigned=True
                break
            elif("which" in childern):
                which_questions.append(question)
                assigned=True
                break
            elif("when" in childern):
                when_questions.append(question)
                assigned=True
                break
    if not assigned:
        rand.append(question)
        for token in doc:
            if("what" == token.text and 'dobj' == token.dep_):
                what_questions.append(question)
                break

print(len(dataset))
print("what : ", len(what_questions))
print("where : ",len(where_questions))
print("why : ",len(why_questions))
print("who : ",len(who_questions))
print("how : ",len(how_questions))
print("which : ",len(which_questions))
print("when : ",len(when_questions))
print(len(what_questions) + len(where_questions) + len(why_questions) + len(who_questions) + len(how_questions) + len(which_questions) + len(when_questions))

9741
what :  5798
where :  2863
why :  242
who :  50
how :  216
which :  44
when :  126
9339


In [4]:
from csense.parser.context import *

def getPos(word, doc):
    for token in doc:
        if(token.text == word):
            return token.pos_

## TODO 
# conjuctions of properties 
# conjuctions of dobj
# conjuctions of pobj
# conjuctions of subject

def parse(doc, questionType, candidates, questionConcept):

    question = Question()
    question.questionType = questionType
    contextMap = {}
    entityToPropMap = {}
    entityToPrepositionPObjMap = {}
    question.candidates = candidates
    question.questionConcept = questionConcept
    
    # find root 
    for token in doc:
        if(token.dep_ == "ROOT"):
            question.root = token.text
            break

    # associate properties
    for token in doc:
        if token.dep_ == 'amod':
            if token.head.text not in entityToPropMap:
                entityToPropMap[token.head.text] = []
            entityToPropMap[token.head.text].append(str(token.text))
            # { "glass" : ["empty", ..]}
            
    # associate prepositions
    for token in doc:
        if token.dep_ == 'prep':
            if token.head.text not in entityToPrepositionPObjMap:
                entityToPrepositionPObjMap[token.head.text] = []
            entities = []
            for child in token.children:
                if(child.pos_ == "NOUN"):
                    ent = Subject(child.text, {})
                if(child.pos_ == "VERB"):
                    ent = Action(child.text, {})
                # associate amods of preposition entities. (rent in(prep) glove-shaped(amod) state(ent))
                if(ent.name in entityToPropMap):
                    if("_" not in ent.properties):
                        ent.properties["_"] = []
                    ent.properties["_"].extend(entityToPropMap[ent.name])
                entities.append(ent)
            entityToPrepositionPObjMap[token.head.text].append((token.text,entities))
            # { "rent" : [(in, [state])]}
            #.   (sub) : [(prep, [pobj])]
    
    # find sub - action pairs
    for chunk in doc.noun_chunks:
        if chunk.root.dep_ == 'nsubj':
            sub = Subject(chunk.root.text, {})
            # associating prop with subject
            if(sub.name in entityToPropMap):
                if("_" not in sub.properties):
                    sub.properties["_"] = []
                sub.properties["_"].extend(entityToPropMap[sub.name])
            
            # associating prepositions with subject
            if(sub.name in entityToPrepositionPObjMap):
                for (prep, pobjs) in entityToPrepositionPObjMap[sub.name]:
                    if(prep not in sub.properties):
                        sub.properties[prep] = []
                    sub.properties[prep].extend(pobjs)
            
            # finding verb
            if(getPos(chunk.root.head.text, doc) == 'VERB'):
                verb = Action(chunk.root.head.text, {})
                
                # associating prop with dobject
                if(verb.name in entityToPropMap):
                    if("_" not in verb.properties):
                        verb.properties["_"] = []
                    verb.properties["_"].extend(entityToPropMap[verb.name])

                # associating prepositions with subject
                if(verb.name in entityToPrepositionPObjMap):
                    for (prep, pobjs) in entityToPrepositionPObjMap[verb.name]:
                        if(prep not in verb.properties):
                            verb.properties[prep] = []
                        verb.properties[prep].extend(pobjs)
                
                
                
                context = Context()
                context.action = verb
                context.subjects.append(sub)
                #todo handle conjunction of subjects
                contextMap[verb.name] = context


    for chunk in doc.noun_chunks:
        # associate dobjs
        if chunk.root.dep_ == 'dobj':
            obj = Object(chunk.root.text, {}) 
            # associating prop with dobject
            if(obj.name in entityToPropMap):
                if("_" not in obj.properties):
                    obj.properties["_"] = []
                obj.properties["_"].extend(entityToPropMap[obj.name])

            # associating prepositions with subject
            if(obj.name in entityToPrepositionPObjMap):
                for (prep, pobjs) in entityToPrepositionPObjMap[obj.name]:
                    if(prep not in obj.properties):
                        obj.properties[prep] = []
                    obj.properties[prep].extend(pobjs)
                    
            if(getPos(chunk.root.head.text, doc) == 'VERB'):
                verb = chunk.root.head.text
                if(verb in contextMap):
                    contextMap[verb].objects.append(obj)

            
    for key in contextMap:
        question.contexts.append(contextMap[key])
    
    return question

In [5]:
doc = nlp("If you have a empty glass and are thirsty, where would you put the glass?")
# doc = nlp("If you rent a condominium in a glove-shaped state where are you?")
parsedSentence = parse(doc, "WHERE",["refrigerator", "table", "dishwasher", "water cooler", "dining room"], "glass")
print(parsedSentence)

root: put,
 contexts: ['subjects: [\'name: you, properties: []\'],action: name: have, properties: [],objects: ["name: glass, properties: [(\'_\', [\'empty\'])]"]', 'subjects: [\'name: you, properties: []\'],action: name: put, properties: [],objects: ["name: glass, properties: [(\'_\', [\'empty\'])]"]'],
 candidates: ['refrigerator', 'table', 'dishwasher', 'water cooler', 'dining room'],
 type: WHERE


In [31]:
#conceptnet getNode API

import requests
import operator
from csense.mining.node import *

def lookup(word):
    url = "http://conceptnet5.media.mit.edu/data/5.4/c/en/" + word + "?offset=0&limit=10000"
    obj = requests.get(url).json()
    print(obj)
    edgeNames = set()
    edges = []
    for edge in obj['edges']:
        # print(edge)
        if edge['start'].get('language') and edge['start']['language'] == 'en' and edge['start']['label'] != word:
            if edge['start']['label'] not in edgeNames:
                node = Edge(edge['start']['label'], edge['weight'], edge['rel']['label'], 'start')
                edges.append(node)
            edgeNames.add(edge['start']['label'])

        elif edge['end'].get('language') and edge['end']['language'] == 'en' and edge['end']['label'] != word:
            if edge['end']['label'] not in edgeNames:
                node = Edge(edge['end']['label'], edge['weight'], edge['rel']['label'], 'end')
                edges.append(node)
            edgeNames.add(edge['end']['label'])

    edges.sort(key=operator.attrgetter('weight'), reverse = True)
    for node in edges:
        print(node)
    print(len(edges))
    return edges
lookup('water_cooler')

{'@context': ['http://api.conceptnet.io/ld/conceptnet5.7/context.ld.json'], '@id': '/c/en/water_cooler', 'edges': [{'@id': '/a/[/r/AtLocation/,/c/en/gossip/,/c/en/water_cooler/]', '@type': 'Edge', 'dataset': '/d/conceptnet/4/en', 'end': {'@id': '/c/en/water_cooler', '@type': 'Node', 'label': 'a water cooler', 'language': 'en', 'term': '/c/en/water_cooler'}, 'license': 'cc:by/4.0', 'rel': {'@id': '/r/AtLocation', '@type': 'Relation', 'label': 'AtLocation'}, 'sources': [{'@id': '/and/[/s/activity/omcs/omcs1_possibly_free_text/,/s/contributor/omcs/guru1/]', '@type': 'Source', 'activity': '/s/activity/omcs/omcs1_possibly_free_text', 'contributor': '/s/contributor/omcs/guru1'}, {'@id': '/and/[/s/activity/omcs/omcs1_possibly_free_text/,/s/contributor/omcs/jlquelch/]', '@type': 'Source', 'activity': '/s/activity/omcs/omcs1_possibly_free_text', 'contributor': '/s/contributor/omcs/jlquelch'}, {'@id': '/and/[/s/activity/omcs/vote/,/s/contributor/omcs/smtango/]', '@type': 'Source', 'activity': '/

In [22]:
# Populating nodeQueue for BFS

nodeQueue = []
questionQueue = []
candidateQueue = []

for candidate in parsedSentence.candidates:
    edges = lookup(candidate.replace(' ','_'))
    doc = nlp(candidate)
    postag = ''
    for token in doc:
        postag = token.pos_
    node = Node(candidate.replace(' ','_'),postag,edges,-1)
    candidateQueue.append(node)
    
for context in parsedSentence.contexts:
    for subject in context.subjects:
        edges = lookup(subject.name.replace(' ','_'))
        doc = nlp(subject.name)
        postag = ''
        for token in doc:
            postag = token.pos_
        node = Node(subject.name.replace(' ','_'),postag,edges,-1)
        questionQueue.append(node)
    
    edges = lookup(context.action.name.replace(' ','_'))
    doc = nlp(context.action.name)
    postag = ''
    for token in doc:
        postag = token.pos_
    node = Node(context.action.name.replace(' ','_'),postag,edges,-1)
    questionQueue.append(node)
    
    for obj in context.objects:
        edges = lookup(obj.name.replace(' ','_'))
        doc = nlp(obj.name)
        postag = ''
        for token in doc:
            postag = token.pos_
        node = Node(obj.name.replace(' ','_'),postag,edges,-1)
        questionQueue.append(node)

addedNodes = set()
for node in questionQueue:
    if node.postag == 'NOUN' and node.name not in addedNodes:
#         node.weight = 1
        nodeQueue.append(node)
        addedNodes.add(node.name)

nodeQueue.sort(key=lambda x: x.weight, reverse=True)
candidateQueue.sort(key=lambda x: x.weight, reverse=True)

for node in nodeQueue:
    print(node)

name: glass
postag: NOUN,
weight: -1,
edges: ['name: window, weight: 8.952764936040708, rel: RelatedTo, type: start', 'name: clear, weight: 6.563840339313565, rel: RelatedTo, type: end', 'name: material, weight: 6.041522986797286, rel: RelatedTo, type: end', 'name: drinking, weight: 5.047771785649585, rel: RelatedTo, type: end', 'name: transparent, weight: 4.846441993875507, rel: RelatedTo, type: end', 'name: windows, weight: 4.814976635457331, rel: RelatedTo, type: end', 'name: drink, weight: 4.302092514114498, rel: RelatedTo, type: start', 'name: sand, weight: 4.262393693689028, rel: RelatedTo, type: end', 'name: cup, weight: 4.134247210799083, rel: RelatedTo, type: end', 'name: container, weight: 3.7825917041097634, rel: RelatedTo, type: end', 'name: pane, weight: 3.179308100829487, rel: RelatedTo, type: end', 'name: window material, weight: 2.85447017850949, rel: RelatedTo, type: end', 'name: see, weight: 2.839718295887815, rel: RelatedTo, type: end', 'name: see through, weight: 2.

In [25]:
# BFS Algorithm

depth = 2
visited = set()

while depth and len(nodeQueue)>0:
    node = nodeQueue.pop(0)
    if node not in visited:
        edges = node.edges
        edges = edges[0:20]
        for edge in edges:
            nodeEdges = lookup(edge.name.replace(' ','_'))
            doc = nlp(edge.name)
            postag = ''
            for token in doc:
                postag = token.pos_
            newNode = Node(edge.name.replace(' ','_'),postag,nodeEdges,-1)
            nodeQueue.append(newNode)
            
        visited.add(node)
        print(len(visited))
        print(len(nodeQueue))
        
    nodeQueue.sort(key=lambda x: x.weight, reverse=True)
    depth -= 1
    
for node in nodeQueue:
    print(node.name)
print(len(nodeQueue))

for ele in visited:
    print(ele.name)
print(len(visited))

1
39
2
58
material
drinking
transparent
windows
drink
sand
cup
container
pane
window_material
see
see_through
through
water
breakable
liquid
glassman
fragile
glass
opening
house
wall
door
panes
pane
shade
through
see
looking
clear
hole
portal
glass_pane
outside
out
square
curtains
view
glass
water
transparent
through
see
see_through
window
distinct
comprehensible
unambiguous
clarity
definite
clearing
transparency
ekphrastic
windowpane_acid
insonorous
shiny
conspicuity
clearedness
58
window
clear
2


In [26]:
# print(where_questions[10]) 
# doc = nlp(question['question']['stem'].lower())

for sent in range(0,len(where_questions)):
# for sent in range(0,20):
    doc = nlp(where_questions[sent]['question']['stem'])

    print(doc)
    print(sent)
    phrases = getSentence(doc)
    for token in doc:
        if(token.dep_ == "ROOT"):
            root = token.text
    print(root)
    print("################")

sent = where_questions[10]['question']['stem']
doc = nlp(sent.lower())
doc = nlp("If you have a empty glass and are thirsty, where would you put the glass?")
# doc = nlp("Where could you see an advertisement while reading news?")
# getParsedSentence(doc)
getSentence(doc)

Sammy wanted to go to where the people were.  Where might he go?
0


NameError: name 'getSentence' is not defined

In [ ]:
def getSentence(doc):
    phrase = []
    sub = []
    verb = []
    obj = []
    phrases = []

    for chunk in doc.noun_chunks:
        if chunk.root.dep_ == 'nsubj':
            
            # if len(sub) == 1:               # tackle 2 subjects one after the other
            #     phrases.append(phrase)
            #     sub = []
            #     verb = []
            #     obj = []
            #     phrase = []

            # sub = chunk.root.text

            sub =  chunk.text
            verb = chunk.root.head.text
            phrase = []
            phrase.append(sub)
            phrase.append(verb)

        elif chunk.root.dep_ == 'dobj' or chunk.root.dep_ == 'pobj':

            # if len(obj) == 1:
            #     phrase.append(obj)
            #     phrases.append(phrase)
            #     verb = []
            #     obj = []
            #     phrase = phrase[0:1]

            obj = chunk.text

            verb = chunk.root.head.text
            if len(phrase) == 2 and phrase[1] != verb:
                phrase[1] += " " + verb
            elif len(phrase) ==1:
                phrase.append(verb)
            
            phrase.append(obj)

            # phrases.append(phrase)
            # obj = []
            # verb = []
            # phrase = phrase[0:1]

        if len(phrase) == 3:
            phrases.append(phrase)
            verb = []
            obj = []
            phrase = phrase[0:1]

        # print(chunk.text, "+", chunk.root.text, "+", chunk.root.dep_, "+", chunk.root.head.text)

    # if len(phrase):
    #     phrases.append(phrase)
    print("PHRASES")
    print(phrases)
    return phrases

In [ ]:
spacy.explain('pobj')

In [ ]:
doc1 = nlp(where_questions[4]['question']['stem'])
doc2 = nlp(where_questions[6]['question']['stem'])
doc3 = nlp(where_questions[9]['question']['stem'])
doc4 = nlp(where_questions[12]['question']['stem'])
doc5 = nlp(where_questions[13]['question']['stem'])
doc6 = nlp(where_questions[16]['question']['stem'])
doc7 = nlp(where_questions[17]['question']['stem'])
doc8 = nlp(where_questions[18]['question']['stem'])
doc9 = nlp(where_questions[19]['question']['stem'])
doc10 = nlp(where_questions[2734]['question']['stem'])
doc11 = nlp(where_questions[2735]['question']['stem'])
doc12 = nlp(where_questions[2743]['question']['stem'])
doc13 = nlp("If you have a empty glass and are thirsty, where would you put the glass?")


In [ ]:
print(doc12)
phrases = getSentence(doc12)
# print(phrases)
print(where_questions[2743])
for token in doc12:
    if(token.dep_ == "ROOT"):
        root = token.text
print(root)

In [ ]:
import requests

def lookup(word):
    url = "http://conceptnet5.media.mit.edu/data/5.4/c/en/" + word
    obj = requests.get(url).json()
    print(obj)
    for edge in obj['edges']:
        meaning = edge['end']['@id'].split('/')
        print(meaning[-1])
        # print(edge['end']['@id'])

In [ ]:
print(parsedSentence)
lookup("glass")